# Training the Change Detection Model

This notebook demonstrates how one can train the change detection model.

In [1]:
# Pytorch imports
import torch
import torchvision
import torchvision.transforms as transforms

# Import dataset that relies on synthetic_anno.json for change detection annotations
from synthetic_data_baselines.datasets.syntheticpairs_dataset import SyntheticPairsDataSet

In [2]:
# This dictionary defines hyperparameters for the dataset
# Complete configurations for each experiment can be found in the configs folder
train_set_params = {
        "dataset_name": "syntheticpairs",
        "root": "/app/renders_multicam_diff_1",
        "loader_params": {
            "batch_size": 16,
            "drop_last": False,
            "pin_memory": True,
            "num_workers": 8
        },
        "mode": "train",
        "crop": True,
        "resize": True,
        "spatial_resolution": [512, 360],
        "overfit": False,
        "normalize": True,
        "augment": True,
        "shuffle": True
    }

trainset = SyntheticPairsDataSet(train_set_params)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=train_set_params["loader_params"]["batch_size"],
                                          shuffle=train_set_params["shuffle"], num_workers=train_set_params["loader_params"]["num_workers"])



In [3]:
# Print number of batches
len(trainloader)

279

In [8]:
# The model is initialized in a similar manner to the dataset
from synthetic_data_baselines.models.changedetection_model import ChangeDetectionModel

model_params = dict({
        "model_name": "changedetection",
        "num_classes": 4,
        "dataset_name": "syntheticpairs",
        "rgb": True, # Use RGB inputs
        "depth": False, # Use depth inputs
        "spatial_resolution": [512, 360],
        "mode": "train",
        "max_epochs": 4000,
        "lr": 0.0004,
        "save_path": "/app/saved_models/changedetection_benchmark/",
        "weights_path": "/app/saved_models/changedetection_benchmark/",
        "load_weights": -1,
        "val_epochs": [1950],
        "val_rate": 50,
        "save_rate": 50,
        "lr_policy": "cosine",
        "lr_decay_iters": 10,
        "batch_size": 16,
        "loss_weights": True
    })
model = ChangeDetectionModel(model_params).model

Loaded weights into backbone
Number total params  39643962


In [9]:
# Print model
model

SingleStream(
  (model): DeepLabV3(
    (backbone): IntermediateLayerGetter(
      (conv1): Conv2d(6, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        

In [15]:
# Parallelize model across GPUs if available
if torch.cuda.is_available():
    model = model.cuda()
    model = torch.nn.DataParallel(model)

# Loss definitions
loss_weights = torch.Tensor([0.0035252888617000786, 0.33161259399877635, 0.3316452266650099, 0.3332168904745137]).cuda()
criterion_loss = torch.nn.CrossEntropyLoss(weight=loss_weights) # Segmentation output loss

# Optimizer definitions
optimizer = torch.optim.SGD(model.parameters(), lr=model_params["lr"])

In [ ]:
# Main training loop over epochs
starting_epoch = 0
num_epochs = model_params['max_epochs']
for epoch in range(starting_epoch, num_epochs):
    train_iterations = len(trainloader)
    train_batch_size = model_params['batch_size']
    
    model.train()
    running_loss = 0
    # Training loop over samples within epoch
    for i, data in enumerate(trainloader):
        # Process data to correct types
        img1, img2, label, scene, depth1, depth2 = data
        img1 = img1.float()
        img2 = img2.float()
        depth1 = depth1.float()
        depth2 = depth2.float()
        label = label.type(torch.LongTensor)
        
        # Move data to GPU if available
        if torch.cuda.is_available():
            img1 = img1.cuda()
            img2 = img2.cuda()
            depth1 = depth1.cuda()
            depth2 = depth2.cuda()
            label = label.cuda()
        
        # Get model output
        model_input = (img1, img2, depth1, depth2)
        model_out = model(model_input)
        
        # Calculate loss function, get gradients, and update network weights
        loss = criterion_loss(self.output, self.label)
        loss.backward()
        running_loss += loss.item()
        for optimizer in self.optimizers:
            optimizer.step()
            optimizer.zero_grad()

    print("Epoch {} Loss: ".format(epoch), running_loss/len(trainloader))

In [ ]:
# Export trained model
torch.save(model.state_dict(), model_params['save_path']+'change_detection_model.pt')